In [ ]:
%load_ext nb_black

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import random as rd
from functools import reduce

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.feature import (
    StringIndexer,
    OneHotEncoder,
    OneHotEncoderEstimator,
    VectorAssembler,
)
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import (
    GBTClassifier,
    GBTClassificationModel,
    RandomForestClassificationModel,
    RandomForestClassifier,
)

from pyspark.ml import evaluation as evals
from pyspark.ml import tuning as tune
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.types import DoubleType

operator, product_name_, model_type = ("cmcc", "jdjk", "click")
model_name = operator + "_" + product_name_ + "_" + model_type
operator_id = {"ctcc": 0, "cmcc": 1, "cucc": 2}.get(operator)
product_name = "('IYB_JDJKZX_BT')"
print(model_name, product_name)

In [ ]:
spark.stop()
spark = (
    SparkSession.builder.appName("zhy_model_{}_prediction".format(model_name))
    .config("spark.dynamicAllocation.maxExecutors", "120")
    .config("spark.executor.memoryOverhead", "12G")
    .config("spark.yarn.executor.memoryOverhead", "12G")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
)

In [ ]:
fake_data = spark.read.parquet(f"/user/zhuyan/model/{model_name}/fake_data")

pipeline_model = PipelineModel.load(f"/user/zhuyan/model/{model_name}/pipeline_model")

gbt_model = GBTClassificationModel.load(f"/user/zhuyan/model/{model_name}/gbt")

model_basic_message = spark.read.parquet(
    f"/user/zhuyan/model/{model_name}/model_basic_message"
)
df_message = model_basic_message.toPandas()

In [ ]:
dpi_symbol = "8_dpi_result" if operator == "cmcc" else "9_dpi_result"
n_visited, n_sent, n_sent_bt, n_host = (
    f"e_n_visited_{dpi_symbol}_modelling",
    f"e_n_sent_{operator}_modelling",
    f"e_n_sent_bt_{operator}_modelling",
    f"e_n_host_{dpi_symbol}_modelling",
)
n_dpi_days = 15

In [ ]:
hosts = str(tuple(df_message.hosts.loc[lambda x: x != "null_value"].tolist()))
spark.sql(
    f"""
select all.rysecret
from (  select phone_number rysecret, myq.host
        from etl_fetch.{dpi_symbol}
        lateral view explode(hosts) q as myq
        where p_biz in ('jrunion')
            AND p_date >= date_format(date_add(current_date(), -40), 'yyyyMMdd')
            AND myq.host IN {hosts}
      union all
      select distinct rysecret, 1 host
        from etl_swap.rp_biz_access_log
        where p_biz in ('loan', 'credit', 'insurance') 
        and dt >= date_sub(current_date(), 180)
    ) all
join (select uid rysecret
        from dw_resources.mapping_uid_property
        where p_operate = {operator_id}
        and p_province not in ('11')
        ) pp on all.rysecret = pp.rysecret
GROUP BY all.rysecret
"""
).createOrReplaceTempView("people_for_prediction")

In [ ]:
label_data = spark.sql(
    f"""
SELECT 

    all.rysecret, 
    n_ins_host_10, n_ins_host_20, n_ins_host_30, fre_ins_host_10, fre_ins_host_20, fre_ins_host_30,
    n_loan_host_10, n_loan_host_20, n_loan_host_30, fre_loan_host_10, fre_loan_host_20, fre_loan_host_30,
    n_credit_host_10, n_credit_host_20, n_credit_host_30, fre_credit_host_10, fre_credit_host_20, fre_credit_host_30,
    n_other_host_10, n_other_host_20, n_other_host_30,  fre_other_host_10, fre_other_host_20, fre_other_host_30,
    ins_host_rate_10, ins_host_rate_20, ins_host_rate_30, ins_fre_rate_10, ins_fre_rate_20, ins_fre_rate_30,
    loan_host_rate_10, loan_host_rate_20, loan_host_rate_30, loan_fre_rate_10, loan_fre_rate_20, loan_fre_rate_30,
    credit_host_rate_10, credit_host_rate_20, credit_host_rate_30, credit_fre_rate_10, credit_fre_rate_20, credit_fre_rate_30,
    n_ins_host_avg_30, n_ins_host_sd_30, n_ins_host_cv_30,
    n_loan_host_avg_30, n_loan_host_sd_30, n_loan_host_cv_30,
    n_credit_host_avg_30, n_credit_host_sd_30, n_credit_host_cv_30,
    n_other_host_avg_30, n_other_host_sd_30, n_other_host_cv_30,
    ins_host_rate_avg_30, ins_host_rate_sd_30, ins_host_rate_cv_30,
    loan_host_rate_avg_30, loan_host_rate_sd_30, loan_host_rate_cv_30,
    credit_host_rate_avg_30, credit_host_rate_sd_30, credit_host_rate_cv_30,
    fre_ins_host_avg_30, fre_ins_host_sd_30, fre_ins_host_cv_30,
    fre_loan_host_avg_30, fre_loan_host_sd_30, fre_loan_host_cv_30,
    fre_credit_host_avg_30, fre_credit_host_sd_30, fre_credit_host_cv_30,
    fre_other_host_avg_30, fre_other_host_sd_30, fre_other_host_cv_30,
    ins_fre_rate_avg_30, ins_fre_rate_sd_30, ins_fre_rate_cv_30,
    loan_fre_rate_avg_30, loan_fre_rate_sd_30, loan_fre_rate_cv_30,
    credit_fre_rate_avg_30, credit_fre_rate_sd_30, credit_fre_rate_cv_30,
    sent_30, sent_90, sent_180, days_since_sent_date,
    click_30, click_90, click_180, days_since_click_date,
    myl_rank, price, age, gender, maker, brand, new_age,
    rank.city_code, province_code, city_level 

FROM people_for_prediction all
LEFT JOIN (
        SELECT *
        FROM bigdata_insurance.{n_host}
        WHERE update_date >= date_format(date_add(current_date(), -{n_dpi_days}), 'yyyyMMdd')
        AND update_date < date_format(date_add(current_date(), -0), 'yyyyMMdd')
        ) n_host on all.rysecret = n_host.rysecret
LEFT JOIN (
        SELECT 
            rysecret, update_date, sent_30, sent_90, sent_180, datediff(update_dt, sent_date) days_since_sent_date,
            click_30, click_90, click_180, datediff(update_dt, click_date) days_since_click_date
        FROM (
            SELECT *, from_unixtime(unix_timestamp(cast(update_date as string), 'yyyyMMdd'), 'yyyy-MM-dd') update_dt
            FROM bigdata_insurance.{n_sent}
            ) sent    
        WHERE update_date >= date_format(date_add(current_date(), -{n_dpi_days}), 'yyyyMMdd')
            AND update_date < date_format(date_add(current_date(), -0), 'yyyyMMdd')
        ) n_sent on all.rysecret = n_sent.rysecret
LEFT JOIN (
        SELECT rysecret, age, gender, price, maker
        FROM bigdata_insurance.e_static_feature_table
        ) st on all.rysecret = st.rysecret
LEFT JOIN (
        SELECT rysecret, brand
        FROM bigdata_insurance.e_brand_table
        ) bt on all.rysecret = bt.rysecret
LEFT JOIN (
        SELECT rysecret, first(cast(rank as int)) myl_rank
        FROM (
            SELECT uid rysecret, rank, p_date, max(p_date) over (partition by uid) max_date
            FROM sample.e_mayilian) sample
        WHERE p_date = max_date
        GROUP BY rysecret
        ) myl on all.rysecret = myl.rysecret
LEFT JOIN (
        SELECT uid rysecret, first(age) new_age
        FROM model_dig.e_user_age_col
        WHERE age is not null
        GROUP BY rysecret
        ) age on all.rysecret = age.rysecret
JOIN (
        SELECT uid rysecret, citycode city_code
        FROM dw_resources.mapping_uid_property
        ) city on all.rysecret = city.rysecret
JOIN (
        SELECT city_id city_code, province_id province_code, city_level_id city_level 
        FROM bigdata_insurance.e_citycode_rank_dict
        ) rank on city.city_code = rank.city_code
"""
).cache()
hosts = str(tuple(df_message.hosts.loc[lambda x: x != "null_value"].tolist()))
dpi_data = spark.sql(
    f"""
SELECT 
    all.rysecret, n_visited.host,
    n_10, n_20, n_30, n_avg_30, n_cv_30,
    fre_10, fre_20, fre_30, fre_avg_30, fre_cv_30
FROM people_for_prediction all
JOIN (
    SELECT *
    FROM bigdata_insurance.{n_visited}
    WHERE update_date >= date_format(date_add(current_date(), -{n_dpi_days}), 'yyyyMMdd')
        AND update_date < date_format(date_add(current_date(), -0), 'yyyyMMdd') 
        AND host IN {hosts}
    ) n_visited ON all.rysecret = n_visited.rysecret 
"""
).cache()

products_click = str(
    tuple(df_message.products_click.loc[lambda x: x != "null_value"].tolist())
)
click_product_data = spark.sql(
    f"""
SELECT 
    all.rysecret, n_sent_bt.product,
    click_30, click_90, click_180, click_360, fre_click_30, fre_click_90, fre_click_180, fre_click_360, days_since_click_date,
    e_ip_30, e_ip_90, e_ip_180, e_ip_360, fre_e_ip_30, fre_e_ip_90, fre_e_ip_180, fre_e_ip_360, days_since_e_ip_date
FROM people_for_prediction all
JOIN (
    SELECT *
    FROM bigdata_insurance.{n_sent_bt}
    WHERE update_date >= date_format(date_add(current_date(), -{n_dpi_days}), 'yyyyMMdd')
        AND update_date < date_format(date_add(current_date(), -0), 'yyyyMMdd')
        AND product IN {products_click} 
    ) n_sent_bt ON all.rysecret = n_sent_bt.rysecret
"""
).cache()

product_sent = str(
    tuple(df_message.products_send.loc[lambda x: x != "null_value"].tolist())
)
sent_product_data = spark.sql(
    f"""
SELECT 
    all.rysecret,  n_sent_bt.product,
    sent_30, sent_90, sent_180, sent_360, days_since_sent_date,
    call_30, call_90, call_180, call_360, days_since_call_date,
    pick_30, pick_90, pick_180, pick_360, days_since_pick_date
FROM people_for_prediction all
JOIN (
    SELECT *
    FROM bigdata_insurance.{n_sent_bt}
    WHERE update_date >= date_format(date_add(current_date(), -{n_dpi_days}), 'yyyyMMdd')
        AND update_date < date_format(date_add(current_date(), -0), 'yyyyMMdd') 
        AND product IN {product_sent}
    ) n_sent_bt ON all.rysecret = n_sent_bt.rysecret 
"""
).cache()

In [ ]:
def get_dummy_data(df, df_message, df_col, df_message_col, label=False):
    """
    df_col, df_message_col为list
    """
    rename_dic = dict(zip(df_message_col, df_col))
    fillna_dic = {"rysecret": "A", "sent_date": "22220202"}
    if label:
        fillna_dic["label"] = 0
    dummy_data = (
        pd.concat(
            [
                pd.DataFrame(columns=df.columns),
                (
                    df_message[df_message_col]
                    .replace("null_value", np.nan)
                    .dropna(how="all")
                    .fillna(method="ffill")
                    .rename(columns=rename_dic)
                ),
            ]
        )
        .fillna(fillna_dic)
        .fillna(-999)
    )
    return dummy_data


label_data = (
    label_data.fillna(
        {
            "maker": "others",
            "brand": "others",
            "price": -999,
            "age": 0,
            "gender": 2,
            "new_age": "null",
            "myl_rank": -999,
        }
    )
    .fillna(
        -999,
        subset=[
            ele
            for ele in label_data.columns
            if ("_cv_" in ele) or ("days_since_" in ele)
        ],
    )
    .fillna(0)
)

dummy_dpi_data = get_dummy_data(dpi_data, df_message, ["host"], ["hosts"])
dpi_data = (
    spark.createDataFrame(dummy_dpi_data)
    .union(dpi_data)
    .filter(F.col("rysecret") != "A")
    .groupBy(["rysecret"])
    .pivot("host")
    .agg(
        *(
            F.first(i).alias(i)
            for i in dpi_data.columns
            if i not in ["rysecret", "host"]
        )
    )
    .fillna(0)
)
print("dpi_data")

dummy_click_product_data = get_dummy_data(
    click_product_data, df_message, ["product"], ["products_click"]
)
click_product_data = (
    spark.createDataFrame(dummy_click_product_data)
    .union(click_product_data)
    .filter(F.col("rysecret") != "A")
    .groupBy(["rysecret"])
    .pivot("product")
    .agg(
        *(
            F.first(i).alias(i)
            for i in click_product_data.columns
            if i not in ["rysecret", "product"]
        )
    )
    .fillna(0)
)
print("click_data")

dummy_sent_product_data = get_dummy_data(
    sent_product_data, df_message, ["product"], ["products_send"]
)
sent_product_data = (
    spark.createDataFrame(dummy_sent_product_data)
    .union(sent_product_data)
    .filter(F.col("rysecret") != "A")
    .groupBy(["rysecret"])
    .pivot("product")
    .agg(
        *(
            F.first(i).alias(i)
            for i in sent_product_data.columns
            if i not in ["rysecret", "product"]
        )
    )
    .fillna(0)
)
print("sent_data")

all_data = (
    reduce(
        lambda df1, df2: df1.join(df2, on=["rysecret"], how="left"),
        [label_data, dpi_data, click_product_data, sent_product_data],
    )
    .withColumn("sent_date", F.lit("20750101"))
    .withColumn("label", F.lit(0))
)
print("all_data")

In [ ]:
for col in [i for i in fake_data.columns if i not in all_data.columns]:
    all_data = all_data.withColumn(col, F.lit(0))

all_data = (
    all_data.select(fake_data.columns).fillna(-999).dropDuplicates(subset=["rysecret"])
)

In [ ]:
piped_all_data = pipeline_model.transform(all_data)

data_pred = gbt_model.transform(piped_all_data)

data_pred = data_pred.select("rysecret", "probability", "prediction")

data_pred.cache()

data_pred.filter(F.col("prediction") == 1).select("rysecret").createOrReplaceTempView(
    "shot"
)

operator_id2 = 8 if operator == "cmcc" else 9
rule_name = f"{operator}_dpi_{operator_id2}_dym_{product_name_ + '_' + model_type}_zhy"
print(rule_name)

spark.sql(
    """
set hive.exec.dynamic.partition.mode=nonstrict
"""
)

In [ ]:
def get_threshold_count(test_pre, prob_col="p_prob"):
    temp = []
    for threshhold in np.arange(0, 1, 0.1):
        num_p = test_pre.filter(F.col(prob_col) >= threshhold).count()
        temp.append((threshhold.round(2), num_p))
    return pd.DataFrame(temp, columns=["prob", "count"])

In [ ]:
vec_to_columns = F.udf(lambda x: x[1].item(), DoubleType())
data_pred1 = data_pred.withColumn("p_prob", vec_to_columns("probability"))

In [ ]:
get_threshold_count(data_pred1)

In [ ]:
(
    data_pred.withColumn("p_prob", vec_to_columns("probability"))
    .filter(F.col("p_prob") >= 0.2)
    .select("rysecret")
    .createOrReplaceTempView("shot")
)
operator_id2 = 8 if operator == "cmcc" else 9
rule_name = f"{operator}_dpi_{operator_id2}_{product_name_ + '_' + 'melo' + '_' + model_type}_zhy"
print(rule_name)

In [ ]:
spark.sql(
    f"""
INSERT OVERWRITE table bigdata_insurance.e_zhuyan_bx_rules_pre partition(rule_name) 
SELECT DISTINCT a.rysecret, b.citycode, b.operator,  b.suffix, b.province, 
        date_add(current_date(), 1) AS p_date, '{rule_name}' AS rule_name
FROM shot AS a 
JOIN dw_resources.mapping_uid_property AS b ON a.rysecret = b.uid 
LEFT JOIN (select rysecret 
            from bigdata_insurance.e_bt_black_list_bx 
            where level = 'jdjk_waihu') AS c ON a.rysecret = c.rysecret 
WHERE c.rysecret is NULL AND b.operator = {operator_id}
"""
)

In [ ]:
spark.sql(
    f"""
select count(1) 
from bigdata_insurance.e_zhuyan_bx_rules_pre
where rule_name = '{rule_name}'
"""
).show()